In [ ]:
import os
import cv2
import time
from tqdm import tqdm
import numpy as np
import pandas as pd
from sklearn.metrics import precision_score, f1_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from keras.layers import Input
from tensorflow.keras import layers, models
from tensorflow.keras.utils import img_to_array, load_img
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Conv2D, GlobalAveragePooling2D, Reshape, MaxPooling2D, UpSampling2D, concatenate
from tensorflow.keras.models import Model
from keras.callbacks import ModelCheckpoint, EarlyStopping

# Define image dimensions and channels
IMG_HEIGHT = 512
IMG_WIDTH = 512
IMG_CHANNELS = 3
input_shape = (IMG_HEIGHT, IMG_WIDTH, 3)
num_classes = 1  # Single class segmentation
SEED = 40

train_image_dir = '/home/syam.varnatt/THESIS/images/train/images'
train_mask_dir = '/home/syam.varnatt/THESIS/images/train/masks'
val_image_dir = '/home/syam.varnatt/THESIS/images/val/images'
val_mask_dir = '/home/syam.varnatt/THESIS/images/val/masks'
SAVE_DIRECTORY = '/home/syam.varnatt/THESIS/attention-unet/aug/results'

def load_dataset(image_dir, mask_dir, img_size):
    images = []
    masks = []
    
    for image_name in tqdm(os.listdir(image_dir)):
        if image_name.lower().endswith(('.jpeg', '.jpg')):
            base_name = os.path.splitext(image_name)[0]
            mask_name = f"{base_name}_mask.jpg"
            
            img_path = os.path.join(image_dir, image_name)
            mask_path = os.path.join(mask_dir, mask_name)
            
            if os.path.exists(img_path) and os.path.exists(mask_path):
                try:
                    image = load_img(img_path, target_size=img_size)
                    mask = load_img(mask_path, target_size=img_size, color_mode="grayscale")
                    
                    image = img_to_array(image)
                    mask = img_to_array(mask)
                    
                    images.append(image)
                    masks.append(mask)
                except Exception as e:
                    print(f"Error processing {image_name}: {e}")
            else:
                print(f"Mask not found for image: {image_name}")
    
    images = np.array(images) / 255.0
    masks = np.array(masks) / 255.0
    masks = (masks > 0.5).astype(np.uint8)
    
    return images, masks

def display_images_and_masks(original_images, original_masks, augmented_images, augmented_masks, num=3):
    """Display original and augmented images and masks in a grid layout."""
    plt.figure(figsize=(15, 10))
    for i in range(num):
        # Display original images
        plt.subplot(4, num, i + 1)
        plt.imshow(original_images[i])
        plt.title(f'Original Image {i+1}')
        plt.axis('off')

        plt.subplot(4, num, i + 1 + num)
        plt.imshow(original_masks[i].squeeze(), cmap='gray')
        plt.title(f'Original Mask {i+1}')
        plt.axis('off')

        # Display augmented images
        plt.subplot(4, num, i + 1 + num*2)
        plt.imshow(augmented_images[i])
        plt.title(f'Augmented Image {i+1}')
        plt.axis('off')

        plt.subplot(4, num, i + 1 + num*3)
        plt.imshow(augmented_masks[i].squeeze(), cmap='gray')
        plt.title(f'Augmented Mask {i+1}')
        plt.axis('off')
    plt.tight_layout()
    plt.show()

# Data augmentation setup
data_gen_args = dict(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='constant',
    cval=0,
    rescale=1.0
)

image_datagen = ImageDataGenerator(**data_gen_args)
mask_datagen = ImageDataGenerator(**data_gen_args)

# Load datasets
train_images, train_masks = load_dataset(train_image_dir, train_mask_dir, (IMG_HEIGHT, IMG_WIDTH))
print("Training dataset loaded completely")
val_images, val_masks = load_dataset(val_image_dir, val_mask_dir, (IMG_HEIGHT, IMG_WIDTH))
print("Validation dataset loaded completely")

# Ensure the same seed is used for images and masks
image_datagen.fit(train_images, augment=True, seed=SEED)
mask_datagen.fit(train_masks, augment=True, seed=SEED)

# Generate augmented images and masks
augmented_images, augmented_masks = [], []
for i in range(len(train_images)):
    img = train_images[i].reshape((1,) + train_images[i].shape)
    mask = train_masks[i].reshape((1,) + train_masks[i].shape)
    
    img_gen = image_datagen.flow(img, batch_size=1, seed=SEED)
    mask_gen = mask_datagen.flow(mask, batch_size=1, seed=SEED)
    
    augmented_images.append(img_gen[0][0])
    augmented_masks.append(mask_gen[0][0])

augmented_images = np.array(augmented_images)
augmented_masks = np.array(augmented_masks)

# Concatenate the original and augmented datasets
all_images = np.concatenate((train_images, augmented_images), axis=0)
all_masks = np.concatenate((train_masks, augmented_masks), axis=0)

total_images = len(all_images)
total_masks = len(all_masks)

print("Total images : ", total_images)
print("Total masks : ", total_masks)
# Display images and masks before and after augmentation
display_images_and_masks(train_images, train_masks, augmented_images, augmented_masks)

# Load datasets
train_images, train_masks = load_dataset(train_image_dir, train_mask_dir, (IMG_HEIGHT, IMG_WIDTH))
print("Training dataset loaded completely")
val_images, val_masks = load_dataset(val_image_dir, val_mask_dir, (IMG_HEIGHT, IMG_WIDTH))
print("Validation dataset loaded completely")

In [ ]:
def attention_block(x, g, inter_channel):
    theta_x = layers.Conv2D(inter_channel, (1, 1), strides=(1, 1), padding='same')(x)
    phi_g = layers.Conv2D(inter_channel, (1, 1), strides=(1, 1), padding='same')(g)
    f = layers.Activation('relu')(layers.add([theta_x, phi_g]))
    psi_f = layers.Conv2D(1, (1, 1), strides=(1, 1), padding='same')(f)
    rate = layers.Activation('sigmoid')(psi_f)
    att_x = layers.multiply([x, rate])
    return att_x

def conv_block(x, filters, kernel_size=(3, 3), padding="same", strides=1):
    conv = layers.Conv2D(filters, kernel_size=kernel_size, padding=padding, strides=strides)(x)
    conv = layers.BatchNormalization()(conv)
    conv = layers.Activation("relu")(conv)
    return conv

def attention_unet(input_shape, num_classes):
    inputs = Input(input_shape)

    # Encoder
    conv1 = conv_block(inputs, 64)
    conv1 = conv_block(conv1, 64)
    pool1 = layers.MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = conv_block(pool1, 128)
    conv2 = conv_block(conv2, 128)
    pool2 = layers.MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = conv_block(pool2, 256)
    conv3 = conv_block(conv3, 256)
    pool3 = layers.MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = conv_block(pool3, 512)
    conv4 = conv_block(conv4, 512)
    pool4 = layers.MaxPooling2D(pool_size=(2, 2))(conv4)

    # Bottleneck
    conv5 = conv_block(pool4, 1024)
    conv5 = conv_block(conv5, 1024)

    # Decoder
    up6 = layers.Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same')(conv5)
    att6 = attention_block(conv4, up6, 512)
    merge6 = layers.concatenate([up6, att6], axis=3)
    conv6 = conv_block(merge6, 512)
    conv6 = conv_block(conv6, 512)

    up7 = layers.Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv6)
    att7 = attention_block(conv3, up7, 256)
    merge7 = layers.concatenate([up7, att7], axis=3)
    conv7 = conv_block(merge7, 256)
    conv7 = conv_block(conv7, 256)

    up8 = layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv7)
    att8 = attention_block(conv2, up8, 128)
    merge8 = layers.concatenate([up8, att8], axis=3)
    conv8 = conv_block(merge8, 128)
    conv8 = conv_block(conv8, 128)

    up9 = layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv8)
    att9 = attention_block(conv1, up9, 64)
    
    merge9 = layers.concatenate([up9, att9], axis=3)
    conv9 = conv_block(merge9, 64)
    conv9 = conv_block(conv9, 64)

    outputs = layers.Conv2D(num_classes, (1, 1), activation='sigmoid')(conv9)

    model = models.Model(inputs=[inputs], outputs=[outputs])

    return model
    
model = attention_unet(input_shape, num_classes)
# Compile the model with appropriate loss and metrics
model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()


In [ ]:
class VisualizationCallback(tf.keras.callbacks.Callback):
    def __init__(self, val_images, val_masks, save_dir, num_samples=3):
        super(VisualizationCallback, self).__init__()
        self.val_images = val_images
        self.val_masks = val_masks
        self.save_dir = save_dir
        self.num_samples = num_samples  # Number of samples to visualize per epoch
        os.makedirs(save_dir, exist_ok=True)  # Ensure the save directory exists

    def on_epoch_end(self, epoch, logs=None):
        # Randomly select a few samples to visualize predictions
        idxs = np.random.choice(len(self.val_images), self.num_samples, replace=False)
        images_to_show = self.val_images[idxs]
        masks_to_show = self.val_masks[idxs]
        
        # Use the model attribute directly, which is set automatically by Keras
        predictions = self.model.predict(images_to_show)

        # Plot and save the images, masks, and predictions
        plt.figure(figsize=(15, 10))
        for i in range(self.num_samples):
            plt.subplot(self.num_samples, 3, i * 3 + 1)
            plt.imshow(images_to_show[i])
            plt.title(f'Input Image {i+1}')
            plt.axis('off')

            plt.subplot(self.num_samples, 3, i * 3 + 2)
            plt.imshow(masks_to_show[i].squeeze(), cmap='gray')
            plt.title(f'True Mask {i+1}')
            plt.axis('off')

            plt.subplot(self.num_samples, 3, i * 3 + 3)
            plt.imshow(predictions[i].squeeze(), cmap='gray')
            plt.title(f'Predicted Mask {i+1}')
            plt.axis('off')

        plt.tight_layout()
        # Save the plot to file
        plt.savefig(os.path.join(self.save_dir, f'epoch_{epoch+1:02d}_predictions.png'))
        plt.close()
        print(f"Saved visualization for epoch {epoch+1}.")

# Instantiate the visualization callback
visualization_callback = VisualizationCallback(val_images, val_masks, SAVE_DIRECTORY)


In [ ]:
class TrainingTimeCallback(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs=None):
        self.start_time = time.time()

    def on_train_end(self, logs=None):
        end_time = time.time()
        training_time = end_time - self.start_time
        print(f"Training Time: {training_time / 60:.2f} minutes")

# Create an instance of the callback
training_time_callback = TrainingTimeCallback()

# Define callbacks
checkpoint = ModelCheckpoint('attention-unet.keras', monitor='val_loss', verbose=1, save_best_only=True, mode='min')
early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='min')

# Train the model
history = model.fit(
    all_images, all_masks,
    validation_data=(val_images, val_masks),
    batch_size=8,
    epochs=150,
    callbacks=[early_stopping, checkpoint, training_time_callback, visualization_callback],
    verbose=1 
)
print("Training completed.")

# Save the training history to an Excel file
history_df = pd.DataFrame(history.history)
history_df.to_excel(os.path.join('training_history.xlsx'), index=False)

In [ ]:

# Predict on validation set
val_preds = model.predict(val_images)

# Convert predictions to binary
val_preds_bin = (val_preds > 0.3).astype(int)

# Calculate additional metrics
precision = precision_score(val_masks.flatten(), val_preds_bin.flatten())
f1 = f1_score(val_masks.flatten(), val_preds_bin.flatten())

# Calculate Mean IoU
mean_iou_metric = tf.keras.metrics.MeanIoU(num_classes=2) 
mean_iou_metric.update_state(val_masks, val_preds_bin)
mean_iou = mean_iou_metric.result().numpy()

# Save metrics to Excel
metrics_df = pd.DataFrame({
    'Precision': [precision],
    'F1 Score': [f1],
    'Mean IoU': [mean_iou]
})
print(f"Precision: ", precision, "F1 score: ", f1, "Mean Iou : ", mean_iou)

# Plot training history
history_df = pd.read_excel('training_history.xlsx')

plt.figure()
plt.plot(history_df['loss'], label='Training Loss')
plt.plot(history_df['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.savefig('loss_plot.png')
plt.show()

plt.figure()
plt.plot(history_df['accuracy'], label='Training Accuracy')
plt.plot(history_df['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.savefig('accuracy_plot.png')
plt.show()

# Save confusion matrix as heatmap
cm = confusion_matrix(val_masks.flatten(), val_preds_bin.flatten())
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", cbar=False)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.savefig('confusion_matrix.png')
plt.show()

# Visualize some predictions on validation set
for i in range(3):
    fig, axs = plt.subplots(1, 3, figsize=(15, 5))
    axs[0].imshow(val_images[i])
    axs[0].set_title('Input Image')
    axs[1].imshow(val_masks[i].squeeze(), cmap='gray')
    axs[1].set_title('Ground Truth')
    axs[2].imshow(val_preds_bin[i].squeeze(), cmap='gray')
    axs[2].set_title('Predicted Mask')
    plt.savefig(f'prediction_{i}.png')
    plt.show()

In [ ]:
#calculate inference time

# Use a small batch from validation data
sample_images = val_images[:5]

# Measure inference time
start_time = time.time()
predictions = model.predict(sample_images)
end_time = time.time()

inference_time = (end_time - start_time) / len(sample_images)
print(f"Average Inference Time per Image: {inference_time:.4f} seconds")

In [ ]:

import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import load_img, img_to_array, array_to_img
from tensorflow.keras.models import load_model

# Define input size
input_size = (512, 512, 3)

# Function to preprocess images
def preprocess_image(image_path, target_size):
    # Load image
    image = load_img(image_path, target_size=target_size[:2])
    # Convert to numpy array
    image_array = img_to_array(image)
    # Normalize the image
    image_array = image_array / 255.0
    return image_array

# Function to save images
def save_image(image_array, save_path):
    # Convert array to image
    image = array_to_img(image_array)
    # Save the image
    image.save(save_path)

# Function to display images
def display_images(images, titles=None, cmap='gray'):
    plt.figure(figsize=(20, 10))
    for i, image in enumerate(images):
        plt.subplot(3, 4, i + 1)  # Adjust depending on the number of images
        plt.imshow(image, cmap=cmap)
        if titles:
            plt.title(titles[i])
        plt.axis('off')
    plt.show()

# Load and preprocess images
image_folder = '/home/syam.varnatt/THESIS/Plot_Images(x10,x20)'
image_paths = [os.path.join(image_folder, fname) for fname in os.listdir(image_folder) if fname.endswith('.png')]

# Sort the image paths by filenames
image_paths = sorted(image_paths)

# Select a subset of images
image_paths = image_paths[0:]

# Predict segmentation masks for each image
# Extract image names from paths
image_names = [os.path.basename(path) for path in image_paths]

# Define the folder path to save images and masks
save_folder = '/home/syam.varnatt/THESIS/attention-unet/aug/results'
os.makedirs(save_folder, exist_ok=True)

# Predict segmentation masks and save images and masks
predicted_masks = []
for i, image_path in enumerate(image_paths):
    # Preprocess image
    image = preprocess_image(image_path, input_size)
    image_batch = np.expand_dims(image, axis=0)
    # Predict mask
    predicted_mask = model.predict(image_batch)[0]
    predicted_mask = (predicted_mask > 0.5).astype(np.uint8)
    
    # Save the original image
    original_image = img_to_array(load_img(image_path, target_size=input_size[:2])) / 255.0
    save_image(original_image, os.path.join(save_folder, f'original_{image_names[i]}'))
    
    # Save the predicted mask
    save_image(predicted_mask, os.path.join(save_folder, f'mask_{image_names[i]}'))
    
    predicted_masks.append(predicted_mask)


In [ ]:
#calculate inference time for predicting heights

# Use a small batch from validation data
sample_images = image_batch[:5]

# Measure inference time
start_time = time.time()
predictions = model.predict(sample_images)
end_time = time.time()

inference_time = (end_time - start_time) / len(sample_images)
print(f"Average Inference Time per flight height predicted Image: {inference_time:.4f} seconds")